In [1]:
import tensorflow as tf
import numpy as np

In [2]:
t = 5
for i in range(t):
    with open("GFSC/reuters_views/view" + str(i) + ".npy", 'rb') as f:
        tmp = tf.cast([np.load(f)], dtype=tf.float32)
        if i == 0:
            X = tf.linalg.matmul(tmp, tmp, transpose_b=True)
        else:
            X = tf.concat([X, tf.linalg.matmul(tmp, tmp, transpose_b=True)], axis=0)
print(X.shape)

(5, 1200, 1200)


In [3]:
steps = 1
alphas, betas, gammas = [1.0], [1000.0], [0.01]
n = X.shape[2]
I = tf.eye(n, dtype=tf.float32)


In [4]:
@tf.function
def fusion(S):
    Z_sum = tf.zeros(S.shape)
    for i in range(t):
        Z = tf.linalg.matmul(
                tf.linalg.inv(X[i] + I * alpha + I * beta * wv[i]),
                S * beta * wv[i] + X[i])
        Z = tf.where(Z[i] > 0, Z, tf.zeros(Z.shape))
        Z = (Z + tf.transpose(Z)) / 2.0
        wv[i].assign(0.5 / tf.norm(Z - S))
        Z *= wv[i]
        Z_sum += Z

    L = tf.linalg.diag(tf.math.reduce_sum(S, 0)) - S
    print('before eigh')
    # (_, F) = tf.linalg.eigh(L)
    F = tf.zeros(S.shape)

    p = tf.norm(F[0][:] - F, axis=1, keepdims=True)
    for i in range(1, n):
         p = tf.concat([p, tf.norm(F[i][:] - F, axis=1, keepdims=True)], 1)
    p = tf.math.square(p) * reg
    S = (Z_sum - p) / tf.math.reduce_sum(wv, 0)
    return S

In [5]:
for a in alphas:
    for b in betas:
        for g in gammas:
            alpha, beta, gamma = a, b, g
            reg = gamma / (4.0 * beta)
            wv = tf.Variable([1.0 / t for _ in range(t)], dtype=tf.float32)
            S = tf.eye(n, dtype=tf.float32)
            for step in range(steps):
                S = tf.where(S > 0, S, tf.zeros(S.shape))
                S = (S + tf.transpose(S)) / 2.0
                tmp = S
                S = fusion(S)
                if step > 5 and tf.norm(S - tmp) / tf.norm(tmp) < 1.e-3:
                    print('Num steps:', step)
                    break
            print(S)
            with open("fusion_matrix/S_reuters.npy", 'wb') as f:
                np.save(f, S.numpy())

before eigh
tf.Tensor(
[[ 9.8387593e-01 -1.3877200e-05 -1.7312734e-05 ...  1.0756154e-05
  -8.0892796e-06  4.9879081e-06]
 [-1.3877200e-05  9.9306440e-01  4.0064751e-05 ... -8.1838897e-07
   3.3399808e-06  2.6084269e-06]
 [-1.7312734e-05  4.0064751e-05  9.8380488e-01 ...  4.6950649e-06
   5.5175570e-07 -1.8874132e-06]
 ...
 [ 1.0756154e-05 -8.1838897e-07  4.6950649e-06 ...  9.9283189e-01
   9.8665259e-06 -1.2301878e-06]
 [-8.0892796e-06  3.3399808e-06  5.5175570e-07 ...  9.8665259e-06
   9.9278432e-01  3.5603061e-06]
 [ 4.9879081e-06  2.6084269e-06 -1.8874132e-06 ... -1.2301878e-06
   3.5603061e-06  9.9515927e-01]], shape=(1200, 1200), dtype=float32)


In [6]:
# x = tf.constant([[[2., 16., 128.],
#                  [4., 32., 256.],
#                  [8., 64., 512.]]], dtype=tf.float32)